In [ ]:
# default_exp followers

In [ ]:
#test_flag_colab
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#hide
!pip install -qU nbdev fastcore
from pathlib import Path
path = Path("/content/drive/My Drive/upwork/Martin followers/tomorrowswig_followers")
%cd "{path}"
from nbdev import * 
from nbdev.showdoc import *
from fastcore.test import *

     |████████████████████████████████| 61kB 3.8MB/s 
/content/drive/My Drive/upwork/Martin followers/tomorrowswig_followers


# Instagram followers
> Here I'll create a script to fetch new followers for an instagram account. The reason is to track followers change by country, facebook API offers only lifetime numbers which are updated daily.

https://developers.facebook.com/tools/explorer/

https://developers.facebook.com/docs/facebook-login/access-tokens/

## Facebook Auth

In [ ]:
#hide
!pip install -qU gspread facebook_business python-facebook-api

     |████████████████████████████████| 1.0MB 4.9MB/s 
     |████████████████████████████████| 10.1MB 18.3MB/s 


In [ ]:
#hide
#test_flag_colab
%%capture

import yaml
with open(path.parent/"facebook_creds.yml") as f:
    data = yaml.safe_load(f)
    %env APP_ID {data.get("APP_ID")}
    %env APP_SECRET {data.get("APP_SECRET")}
    %env TOKEN {data.get("TOKEN")}
    %env USER_ID {data.get("USER_ID")}

In [ ]:
#export
import os
from datetime import datetime, timedelta
from typing import *

import pandas as pd
import numpy as np
from pyfacebook import IgProApi
from pyfacebook.error import *
from tomorrowswig_followers.core import *
from tomorrowswig_followers.ads import *

In [ ]:
APP_ID = os.environ.get("APP_ID")
APP_SECRET = os.environ.get("APP_SECRET")
TOKEN = os.environ.get("TOKEN")
USER_ID = os.environ.get("USER_ID")

## Get new followers with graph api

In [ ]:
#hide
!curl -i -X GET \
 "https://graph.facebook.com/v7.0/17841409206135869/insights?metric=audience_country&period=lifetime&access_token={TOKEN}"

HTTP/2 200 
vary: Accept-Encoding
etag: "4ca7035649064da000a39d9342473fa3157d045c"
x-business-use-case-usage: {"17841409206135869":[{"type":"instagram","call_count":1,"total_cputime":1,"total_time":1,"estimated_time_to_regain_access":0}]}
content-type: application/json; charset=UTF-8
facebook-api-version: v7.0
strict-transport-security: max-age=15552000; preload
pragma: no-cache
x-fb-rev: 1002505327
access-control-allow-origin: *
cache-control: private, no-cache, no-store, must-revalidate
x-fb-trace-id: BiWhncYz5DW
x-fb-request-id: ASj4eBp4ydcmsDi1mERkZk9
expires: Sat, 01 Jan 2000 00:00:00 GMT
x-fb-debug: enrFGGhWjae3mSW5W/X1gnEnQuEXtBgZLEh99aPNsVqwv7Kt25cQuJZT9F5URKwhu5U+zMWs8+cBNRxc5A4Gew==
date: Thu, 13 Aug 2020 01:34:55 GMT
alt-svc: h3-29=":443"; ma=3600,h3-27=":443"; ma=3600

{"data":[{"name":"audience_country","period":"lifetime","values":[{"value":{"RS":56,"DE":939,"NP":37,"BD":53,"TW":125,"PT":155,"JO":41,"JP":83,"UA":39,"FR":56,"HU":99,"BR":2454,"MA":85,"SG":707,"DZ":104,"KE":

In [ ]:
api = IgProApi(app_id=APP_ID, app_secret=APP_SECRET, long_term_token=TOKEN,
               version="5.0")

In [ ]:
try:
    response = api.get_user_insights(user_id=USER_ID, period="lifetime", metrics=["audience_country"],
                                    return_json=True)
except PyFacebookException as e:
    display(e.message)

In [ ]:
response = response[0]["values"][0]

In [ ]:
response["end_time"]

'2020-08-12T07:00:00+0000'

In [ ]:
#export
def get_followers() -> Tuple[str, Dict[str, int]]:
    api = IgProApi(
        app_id=APP_ID, app_secret=APP_SECRET, long_term_token=TOKEN, version="5.0"
    )
    try:
        response = api.get_user_insights(
            user_id=USER_ID,
            period="lifetime",
            metrics=["audience_country"],
            return_json=True,
        )[0]["values"][0]
        return response["end_time"].split("T")[0], response["value"]
    except PyFacebookException as e:
        return (e.message, {0: 0})

In [ ]:
#export
def get_new_followers() -> Tuple[str, Dict[str, int]]:
    api = IgProApi(
        app_id=APP_ID, app_secret=APP_SECRET, long_term_token=TOKEN, version="5.0"
    )
    try:
        response = api.get_user_insights(
                user_id=USER_ID,
                period="day",
                metrics=["follower_count"],
                return_json=True,
            )[0]["values"][0]
        return response["end_time"].split("T")[0], response["value"]
    except PyFacebookException as e:
        return (e.message, 0)

In [ ]:
get_new_followers()

('2020-08-11', 84)

## Google Sheets

### Insert new followers to a google sheet 

In [ ]:
end_time, followers = get_followers()

In [ ]:
df = get_df("History")

In [ ]:
end_time = (datetime.strptime(end_time, "%Y-%m-%d") - timedelta(days=1)).strftime("%b %d %Y")

In [ ]:
new_followers = pd.Series(followers)
new_followers.name = f"{end_time} {str(datetime.utcnow()).split('.')[0]}"
last_entry = df.iloc[:, 0]

In [ ]:
if not np.array_equal(last_entry[last_entry != 0].values, new_followers.sort_index().values):
    df = pd.concat([df, new_followers], axis=1)
    df = df[[df.columns[-1]] + df.columns[:-1].tolist()]
df.sort_index(inplace=True)

In [ ]:
df = df.fillna(0).astype(int)

In [ ]:
#export
def get_updated_followers(
    df: pd.DataFrame, data: Dict[str, int], end_time: str
) -> pd.DataFrame:
    new_followers = pd.Series(data)
    date = (datetime.strptime(end_time, "%Y-%m-%d") - timedelta(days=1)).strftime(f"%b %d %Y{' '*16}")
    new_followers.name = f"{date} {str(datetime.utcnow()).split('.')[0]}"
    last_entry = df.iloc[:, 0]
    if np.array_equal(last_entry[last_entry != 0].values, new_followers.sort_index().values):
        return pd.DataFrame()
    else:
        df = pd.concat([df, new_followers], axis=1)
        df = df[[df.columns[-1]] + df.columns[:-1].tolist()]
        df.index.name = "countries" 
        df.sort_index(inplace=True)
        return df.fillna(0).astype(int)

In [ ]:
fetched_followers = {'AR': 1001, 'AU': 724, 'BD': 55, 'BR': 2250, 'CA': 1680, 'CL': 526, 'CO': 6}
end_time = "2020-07-21"
old_followers = {'AR': 100, 'AU': 70, 'BD': 0, 'BR': 0, 'CA': 0, 'CL': 0, 'CO': 0}
followers_df = pd.DataFrame.from_dict(old_followers, orient="index")

test_followers_df = get_updated_followers(followers_df, fetched_followers, end_time)

test_followers_df.rename(columns={test_followers_df.columns[0]: "2020-01-01" }, inplace=True)
assert {0: old_followers, '2020-01-01': fetched_followers, } == test_followers_df.to_dict()

test_followers_df = get_updated_followers(followers_df, fetched_followers, end_time)
test_followers_df.rename(columns={test_followers_df.columns[0]: "2020-01-02" }, inplace=True)
assert {0: old_followers, '2020-01-02': fetched_followers} == test_followers_df.to_dict()

In [ ]:
#export
def get_dif(df: pd.DataFrame) -> pd.DataFrame:
    change_df = df.diff(axis=1, periods=-1)
    change_df = change_df.fillna(0).astype(int)
    return change_df

In [ ]:
#export
def get_followers_change(history_df: pd.DataFrame) -> pd.DataFrame:
    new_followers = get_dif(history_df)
    new_followers = new_followers.iloc[:,[0]].replace(0, np.nan)
    new_followers = new_followers.dropna(axis=1, how="all").iloc[:, :1].dropna().astype(int)
    return new_followers

In [ ]:
#export
def add_followers(df: pd.DataFrame, new_followers: pd.DataFrame):
    df["Ads Followers Change"] = new_followers
    df.fillna(0, inplace=True)
    df["% (Clicks)"] = (df["Ads Followers Change"] / df["Clicks (All)"])
    df["% (Impressions)"] = (df["Ads Followers Change"] / df["Impressions"])
    df["Cost Per Follow"] = (df["Amount Spent (USD)"] / df["Ads Followers Change"])
    df.replace([np.inf], 0, inplace=True)
    df["% (Impressions)"] = df["% (Impressions)"].round(4)
    df[df.columns.difference(["% (Impressions)"])] = df[df.columns.difference(["% (Impressions)"])].round(2)

random followers + followers from ads - unfollowers = followers gain

unfollowers = random followers + followers from ads - followers gain = new followers - followers gain

random_followers = new followers - followers from ads

In [ ]:
#export
def more_stats(df: pd.DataFrame, followers_change: pd.DataFrame) -> pd.DataFrame:
    stats = pd.Series(dtype=float, name="Followers")
    ads_followers_gain = df["Ads Followers Change"].sum()
    followers_gain = followers_change.sum().item()
    spent = df.loc["total", "Amount Spent (USD)"]
    stats["Ads Followers Change"] = ads_followers_gain
    new_followers = get_new_followers()[1]
    stats["Random Followers"] = new_followers - ads_followers_gain
    stats["New Followers"] = new_followers
    stats["Unfollowers"] = stats["New Followers"] - followers_gain
    stats["Followers Change"] = ads_followers_gain + stats["Random Followers"] - stats["Unfollowers"]

    stats_df = pd.DataFrame(stats)
    stats_df.index.name = "  "
    stats_df["Conversion Rate"] = stats_df["Followers"] / df.loc["total", "Clicks (All)"]
    stats_df[" "] = 0
    stats_df["Cost Per Follower/Increase"] = spent/ stats_df["Followers"]
    stats_df.iloc[1:, 1] = 0
    stats_df.iloc[[1,3,4], 3] = 0
    return stats_df.round(2)

In [ ]:
#export
def update_insights(history_df: pd.DataFrame, date:str):
    followers_change = get_followers_change(history_df)
    df = get_df(date)
    if df.empty:
        create_insights()
        df = get_df(date)
    if "PERFORMANCE:" in df.index:
        cut_off = df.reset_index().index[df["Ad Name"] == "Ads Followers Change"].item()
        df = df[:cut_off - 3]
    worksheet_name = date
    add_followers(df, followers_change)

    stats_df = more_stats(df, followers_change)
    empty = pd.Series(name="", dtype=str)
    df = df.append([empty] * 4)
    write_df(df, worksheet_name)
    stats_df = stats_df.append([empty] * 15)
    write_df(stats_df, worksheet_name, loc=f"B{len(df)}")
    notes_df = pd.DataFrame([["", "CONSIDER:", "", "", "TO DO:"]], index=["PERFORMANCE:"], columns=[""] * 5)
    write_df(notes_df, worksheet_name, loc=f"A{len(stats_df) + len(df) - 14}")

    wsh = get_worksheet(worksheet_name)
    unf_cell = wsh.find("Unfollowers")
    wsh.format(f"C{wsh.find('Unfollowers').row}", {"textFormat": {
        "foregroundColor": {
            "red": 0.0,
            "green": 0.0,
            "blue": 1.0
        }}},)

In [ ]:
#export
def save_followers() -> Tuple[pd.DataFrame, str]:
    df = get_df("History")
    end_time, followers = get_followers()
    df = get_updated_followers(df, followers, end_time)
    if df.empty:
        return end_time, "No followers change"
    date = df.columns[0].split("  ")[0]
    update_insights(df, date)
    write_df(df, "History")
    return date, f"Updated followers and insights for '{date}'"

In [ ]:
save_followers()

('2020-08-12', 'No followers change')

### Get difference from the previous day

In [ ]:
df = get_df("History")

In [ ]:
change_df = df.diff(axis=1, periods=-1)

In [ ]:
change_df = change_df.fillna(0).astype(int)

In [ ]:
mask = (change_df != 0).any(axis=0)

In [ ]:
change_df = change_df.loc[:, mask]

In [ ]:
change_df = change_df.applymap(lambda x: f"+{x}" if x >0 else x)

In [ ]:
change_df = "(" + change_df.astype(str) + ")" + " " + df.loc[:, mask].astype(str)

In [ ]:
change_df = change_df.replace(["\(0\) 0", "\(0\) "], "", regex=True)

In [ ]:
change_history = get_worksheet("Change History")

In [ ]:
#export
def make_change(df: pd.DataFrame) -> pd.DataFrame:
    change_df = df.diff(axis=1, periods=-1)
    change_df = change_df.fillna(0).astype(int)
    zeros_mask  = (change_df != 0).any(axis=0)
    change_df = change_df.loc[:, zeros_mask]
    change_df = change_df.applymap(lambda x: f"+{x}" if x > 0 else x)
    change_df = "(" + change_df.astype(str) + ")" + " " + df.loc[:, zeros_mask].astype(str)
    return change_df.replace(["\(0\) 0", "\(0\) "], "", regex=True)

In [ ]:
#export
def save_change():
    df = get_df("History")
    change_df = make_change(df)
    write_df(change_df, "Change History")

In [ ]:
# save_change()

In [ ]:
test_change_df = make_change(pd.DataFrame({"2020-07-21": [1001, 724, 55, 2250, 1680],
                                           "2020-07-20": [1004, 708, 55, 2300, 1600],
                                           "2020-07-19": [1004, 708, 55, 2300, 1600]},
                                          index=["AR", "AU", "BD", "BR", "CA"]))
assert test_change_df.iloc[4,:].to_list() == ["(+80) 1680"]
assert test_change_df.iloc[3,:].to_list() == ["(-50) 2250"]

In [ ]:
#export
def update(event: Dict = None, context=None,):
    get_followers()
    date, message = save_followers()
    save_change()
    return message

In [ ]:
update()

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_followers.ipynb.
Converted 02_ads.ipynb.
Converted index.ipynb.
